Doc2vec model with small finetuning built on the dataset of 35k quotes


In [1]:
#preprocessing libriaris
import nltk
nltk.download("stopwords")
#stopwords modules
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

import pandas as pd
import numpy as np
import io

import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function (stopwords, puncuation, etc)
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != " " \
              and token != "\n" \
              and all((c not in token.strip()) for c in punctuation)] 
    text = " ".join(tokens)
    
    return text


In [18]:
#loading dataset
qoutes_path = '../parsing/data/big_data.csv'

df_35k = pd.read_csv(qoutes_path, sep=',', engine ='python') #reading quotes to the dataframe
#with open(qoute_path, 'r', encoding='utf-8') as file:
#    data = file.read().replace('\n', ' nextquote ')
#    processed_data = preprocess_text(data)  #preprocessing qoutes
#dataframe with quotes and their processed versions
#df.insert(loc=1, column="processed_quotes", value=processed_data.split('nextquote')[:-1], allow_duplicates=False) 
df_35k

,Unnamed: 0,Unnamed: 0.1,id,citation,author,raw_tags,tags,strlen,num_words
0,0,0,0,Предпочитая быть романтиком во время тягостны...,Игорь Губерман,"['юмор', 'олюбвисулыбкой']","['юмор', 'олюбвисулыбкой']",112,17
1,1,1,1,"Чужую беду ощущая своей, вживаясь в чужие печа...",Игорь Губерман,"['ирония', 'стихи']","['ирония', 'стихи']",116,19
2,2,2,2,"Обманчива наша земная стезя, Идешь то туда, то...",Игорь Губерман,"['жизнь', 'стихи', 'осознание']","['жизнь', 'стихи', 'осознание']",122,22
3,3,3,3,"Жить, покоем дорожа, — пресно, тускло, просток...",Игорь Губерман,"['жизнь', 'стихи', 'душа']","['жизнь', 'стихи', 'душа']",103,15
4,4,4,4,"Все предрассудки прочь отбросив, Но чтоб от Бо...",Игорь Губерман,"['стихи', 'предрассудкииразум']","['стихи', 'предрассудкииразум']",111,19
...,...,...,...,...,...,...,...,...,...
35004,35004,35004,35004,"Каждый имеет в сердце свое «кладбище», куда ни...",Чингиз Абдуллаев,"['боль', 'сомнения', 'страхи', 'сердце']","['боль', 'сомнения', 'страхи', 'сердце']",116,20
35005,35005,35005,35005,Мужчина и женщина равны. Абсолютно равны. Полн...,Чингиз Абдуллаев,"['отношения', 'чувства']","['отношения', 'чувства']",177,29
35006,35006,35006,35006,"Для женщины важен сам факт, что ей хотят сдела...",Чингиз Абдуллаев,"['отношения', 'цитаты']","['отношения', 'цитаты']",164,28
35007,35007,35007,35007,"Для мужчины нормально, если он защищает женщин...",Чингиз Абдуллаев,"['жизнь', 'женщины', 'мужчина']","['жизнь', 'женщины', 'мужчина']",248,37


In [43]:
df_35k.rename(columns={'citation': 'quote'}, inplace=True)
df_35k['quote'].to_csv('processed_data/pure_q_35k.csv', index=False)
df = pd.read_csv('processed_data/pure_q_35k.csv', sep='<\d*>', engine ='python') #reading quotes to the dataframe
with open('processed_data/pure_q_35k.csv', 'r', encoding='utf-8') as file:
    data = file.read().replace('\n', ' nextquote ')
    processed_data = preprocess_text(data)  #preprocessing qoutes
#dataframe with quotes and their processed versions
df.insert(loc=1, column='processed_quote', value=processed_data.split('nextquote')[1:-1], allow_duplicates=False) 
df

,quote,processed_quote
0,"""Предпочитая быть романтиком во время тягостн...",предпочитать романтик время тягостный реше...
1,"""Чужую беду ощущая своей, вживаясь в чужие печ...",чужой беда ощущать свой вживаться чужой печал...
2,"""Обманчива наша земная стезя, Идешь то туда, т...",обманчивый наш земной стезя идти туда обратно...
3,"""Жить, покоем дорожа, — пресно, тускло, просто...",жить покой дорожить пресный тускло простокваш...
4,"""Все предрассудки прочь отбросив, Но чтоб от Б...",весь предрассудок прочь отбрасывать бог секре...
...,...,...
35004,"""Каждый имеет в сердце свое «кладбище», куда н...",каждый иметь сердце свой « кладбище никто пу...
35005,"""Мужчина и женщина равны. Абсолютно равны. Пол...",мужчина женщина равный абсолютно равный полно...
35006,"""Для женщины важен сам факт, что ей хотят сдел...",женщина важный факт хотеть сделать приятный м...
35007,"""Для мужчины нормально, если он защищает женщи...",мужчина нормально защищать женщина любить всп...


In [44]:
df.to_csv('processed_data/pure_q_35k.csv', index=False)

In [3]:
df_pure35k = pd.read_csv('processed_data/pure_q_35k.csv', sep=',', engine ='python')
df_pure35k

,quote,processed_quote
0,"""Предпочитая быть романтиком во время тягостн...",предпочитать романтик время тягостный реше...
1,"""Чужую беду ощущая своей, вживаясь в чужие печ...",чужой беда ощущать свой вживаться чужой печал...
2,"""Обманчива наша земная стезя, Идешь то туда, т...",обманчивый наш земной стезя идти туда обратно...
3,"""Жить, покоем дорожа, — пресно, тускло, просто...",жить покой дорожить пресный тускло простокваш...
4,"""Все предрассудки прочь отбросив, Но чтоб от Б...",весь предрассудок прочь отбрасывать бог секре...
...,...,...
35004,"""Каждый имеет в сердце свое «кладбище», куда н...",каждый иметь сердце свой « кладбище никто пу...
35005,"""Мужчина и женщина равны. Абсолютно равны. Пол...",мужчина женщина равный абсолютно равный полно...
35006,"""Для женщины важен сам факт, что ей хотят сдел...",женщина важный факт хотеть сделать приятный м...
35007,"""Для мужчины нормально, если он защищает женщи...",мужчина нормально защищать женщина любить всп...


In [11]:
quote_tokens = [q.split() for q in df_pure35k['processed_quote'].values.tolist()]
quote_words = [q for q in df_pure35k['quote'].values.tolist()]
#quote_tokens[0], quote_words[0]

In [5]:
#preparing data for doc2vec
def tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
quotes_for_training = list(tagged_document(quote_tokens))

In [38]:
#buiding a model
#model = gensim.models.doc2vec.Doc2Vec(vector_size=50, dm=1, window=4, min_count=1, negative=5, epochs=30, workers=4)
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, window=8, epochs=30, workers=4)

model.build_vocab(quotes_for_training)
#model.intersect_word2vec_format('ruscorpora_mean_hs.model.bin', binary=True)
model.train(quotes_for_training, total_examples=model.corpus_count, epochs=model.epochs)

In [54]:
model.save('models/d2v_35k_exp.model')

In [6]:
model_35k_exp = gensim.models.doc2vec.Doc2Vec.load('models/d2v_35k_exp.model')

In [8]:
#request handling
def request_processing():
    request = preprocess_text(input()).split()
    return request

In [14]:
def cluster_check(sim, df, model):
    avg = sum([sim_coef for docid, sim_coef in sim])/len(sim)
    cluster_coef=[]
    for q in sim:
        q_sim = model.dv.most_similar(model.infer_vector(df['processed_quote'][q[0]].split()), topn=len(df))
        q_dict = {}
        q_dict = dict(q_sim)
        q_sim_sim = [q_dict[docid] for docid, sim_coef in sim if docid != q[0]]
        cluster_coef.append((q[0],sum(q_sim_sim)/len(q_sim_sim)))
    return avg, cluster_coef

In [42]:
Q_NUMBER=10
request = request_processing()

sim = model.dv.most_similar(model.infer_vector(request, epochs=30), topn=Q_NUMBER)
avg, cluster_coefs = cluster_check(sim, df_pure35k, model)
cluster_dict = dict(cluster_coefs)
print(avg)
for quote in sim:
    print(quote_words[quote[0]], round(quote[1], 3), round(cluster_dict[quote[0]], 3), '\n', df_35k['author'][quote[0]])

print("--------------------------------------------vs old---------------------------------------------------")


sims = model_35k_exp.dv.most_similar(model_35k_exp.infer_vector(request), topn=Q_NUMBER)
avg, cluster_coefs = cluster_check(sims, df_pure35k, model_35k_exp)
cluster_dict = dict(cluster_coefs)
print(avg)
for quote in sims:
    print(quote_words[quote[0]], round(quote[1], 3), round(cluster_dict[quote[0]], 3))


умные люди благоразумно помалкивают
0.8932140648365021
Если долго смотреть в бездну-бездна начинает смотреть на тебя 0.899 0.936 
 Фридрих Ницше
"Я умный, а ты - молодой." 0.898 0.933 
 Юлиан Семенов
"Раз я знаю, что ты придешь, я могу тебя долго ждать." 0.895 0.932 
 Альбер Камю
"«Нет страшнее одиночества, чем одиночество вдвоем…»" 0.895 0.923 
 Омар Хайям
"Он не мог подолгу смотреть на нее, как на солнце, но видел ее, как солнце, и не глядя" 0.892 0.908 
 Лев Николаевич Толстой
"Меня потрясло не то, что ты меня обманываешь, а то, что я тебе больше не верю." 0.892 0.913 
 Фридрих Ницше
"Несчастье большинства людей в том, что они считают себя способными на большее, чем могут." 0.892 0.922 
 Максим Горький
"Постарайтесь получить то, что хотите, или же вы вынуждены будете получить то, что получили." 0.891 0.923 
 Джордж Бернард Шоу
"Умными мы считаем тех, кто с нами согласен." 0.889 0.895 
 Михаил Веллер
"Бог создал нас без нас, но спасти нас без нас не сможет." 0.888 0.921 
 Аврелий Авг

In [43]:
preprocess_text("умные люди благоразумно помалкивают")

'умный человек благоразумно помалкивать'

In [45]:
len(model.dv)

35009